In [0]:
%scala
val checks_input_features_dataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.checks_input_features
""")

println(s"checks_input_features_dataset count: ${checks_input_features_dataset.count}")

In [0]:
%scala

val checks_results_features_dataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.checks_results_features
""")

display(checks_results_features_dataset)

In [0]:
%scala

var grouped_check_names = checks_results_features_dataset.groupBy("checkName").count()

display(grouped_check_names)

In [0]:
%scala
grouped_check_names = checks_results_features_dataset.groupBy("checkName", "isFailed", "comment").count()

display(grouped_check_names)

In [0]:
%scala
grouped_check_names = checks_results_features_dataset.groupBy("checkName", "isFailed").count()

display(grouped_check_names)

In [0]:
%scala
grouped_check_names = checks_results_features_dataset.groupBy("checkName", "isFailed", "comment").count()

display(grouped_check_names)

In [0]:
%scala

val aptInsideAerodromeDS = checks_results_features_dataset
  .filter("checkName == 'APT_INSIDE_AERODROME' AND comment LIKE 'APT located%'")


println(s"aptInsideAerodromeDS: ${aptInsideAerodromeDS.count}")
display(aptInsideAerodromeDS)

In [0]:
%scala
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions._

val aerodromeFailedAptDS = checks_input_features_dataset.as("apt")
                .join(aptInsideAerodromeDS.as("aptOnAerodrome"), col("apt.featureId").equalTo(col("aptOnAerodrome.featureId")), "inner")
                .withColumn("aeroddromid", 
  regexp_extract(col("comment"), "APT located (inside aerodrome|on runway): (.+)", 2)
)
                .select(col("apt.featureId"),
                col("apt.apt.location"),
                col("aptOnAerodrome.comment"),
                col("aeroddromid"),
                col("isFailed"),
                )

display(aerodromeFailedAptDS)
                

In [0]:
%scala
import org.apache.spark.sql.functions.col
import com.tomtom.orbis.io.spark.model.Id
import java.lang.Long

def convertOrbisIdToString(orbisId: Id): String = {
  val COLON_SEPARATOR = ":"
  Seq(orbisId.layerId.getOrElse(0).toString, 
      Long.toUnsignedString(orbisId.high), 
      Long.toUnsignedString(orbisId.low)).mkString(COLON_SEPARATOR)
}

// UDF to apply the function to the DataFrame
val convertOrbisIdUDF = udf((orbisId: Id) => convertOrbisIdToString(orbisId))

val aerodrome_dataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.layer_19174
""").withColumn("orbisIdString", convertOrbisIdUDF(col("id")))
.filter("id.layerId != null")
.select(col("id"),col("orbisIdString"),col("wkt"))

display(aerodrome_dataset)

In [0]:
%scala
import org.apache.spark.sql.functions.col

val failedAPTWithAerodrome = aerodromeFailedAptDS.as("add").join(aerodrome_dataset.as("pdd"), col("add.aeroddromid") === col("pdd.orbisIdString"), "left").select(col("featureId"),
col("location"),
col("comment"),
col("isFailed"),
col("aeroddromid"),
col("pdd.wkt"))

display(failedAPTWithAerodrome)